<a href="https://colab.research.google.com/github/CruiseTian/CloabNotebook/blob/master/Wizard_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <font size=5px color="red">✦ *Google Colab 突破90分钟自动断开:</font>
<p><font size=3px > 每60分钟自动运行代码以刷新90分钟断开限制. 打开 developer-settings (在你的浏览器) 快速健 Ctrl+Shift+I 然后按console 输入以下代码 Enter. ( mac 按 Option+Command+I)</p><b>复制以下隐藏代码粉贴在浏览器console！！不要关闭浏览器以免失效</b>

<code>function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-connect-button").click() 
}setInterval(ClickConnect,6000)</code>

In [ ]:
#@markdown <h3>← 输入了代码后运行以防止断开</h>


import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

## **准备工作**

+ SA账号配置**请将SA文件夹压缩为account.zip并放入谷歌个人云盘不是团队盘**,然后运行命令挂载谷歌云盘并配置SA<br>
+ 将rclone的配置文件**rclone.conf放入谷歌个人云盘不是团队盘**
+ 将redis的配置文件 **redis.conf** 和修改好的**config.properties**文件放入**谷歌个人云盘的<font size=4px color="#95C7AE">wizard文件夹下</font>**，<font size=4px color="red">注意不是团队盘</font>

## **环境配置**

In [ ]:
#@markdown <h3>← 挂载谷歌云盘并配置SA</h>
!rm -rf /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Shanghai /etc/localtime
print("时区修改成功")
from google.colab import drive
drive.mount("/content/drive")
print("磁盘挂载成功！")
!mkdir /root/accounts
!rm -rf /root/accounts/*
print("/root/accounts/目录下sa总数为：")
!ls -l | grep "^-" | wc -l
print("已删除旧的SA。")
!cp "/content/drive/My Drive/account.zip" "/root/accounts/"
!unzip -d /root/accounts /root/accounts/account.zip
from IPython.display import HTML, clear_output
clear_output()
print("SA配置完成。")
%cd /root/accounts/
print("/root/accounts/目录下sa总数为：")
!ls -l | grep "^-" | wc -l
#!rclone copy "/content/drive/My Drive/sa2/" "/content/AutoRclone/accounts/"

In [ ]:
#@markdown <h3>运行安装fclone<h>
#!wget https://raw.githubusercontent.com/cgkings/fclone_shell_bot/master/fclone/fclone.zip && unzip fclone.zip && mv fclone /usr/bin && chmod +x /usr/bin/fclone && fclone version
!bash <(wget -qO- https://git.io/JJYE0)
!cp "/content/drive/My Drive/rclone.conf" "/root/.config/rclone/"
print("Successfully Installed")

In [ ]:
#@markdown <h3>运行配置redis<h>
from IPython.display import HTML, clear_output
%cd /content/
print("开始配置redis")
!rm -rf redis*
!wget https://download.redis.io/releases/redis-5.0.4.tar.gz 
!tar -zxvf redis-5.0.4.tar.gz 
%cd redis-5.0.4
!make
clear_output()
!cp -f "/content/drive/My Drive/wizard/redis.conf" "/content/redis-5.0.4"
print("redis配置完成")
%cd src/
!./redis-server /content/redis-5.0.4/redis.conf
print("redis成功启动")

In [ ]:
#@markdown <h3>运行配置wizard<h>
%cd /content/
print("开始配置wizard")
!rm -rf wizard_bot*
!wget https://github.com/swordman-sm/wizard_bot/releases/download/wizard_bot_v0.11_7/wizard_bot_v0.11_7.zip
!unzip wizard_bot_v*.zip
!rm -f wizard_bot_v*.zip
!mv /content/wizard_bot_v* /content/wizard_bot
!cp -f "/content/drive/My Drive/wizard/config.properties" "/content/wizard_bot/config"
clear_output()
print("wizard配置完成")

## **运行wizard Bot**

In [ ]:
#@markdown <h3>运行以启动wizard bot<h>
!apt-get install screen
from IPython.display import HTML, clear_output
clear_output()
!screen --version
print("成功安装screen")
%cd /content/wizard_bot
!screen -dmS wizard java -jar wizard.jar
print("wizard bot已启动")

In [ ]:
#@markdown <h3>运行以查看wizard bot日志<h>
!tail -f /content/wizard_debug.log

In [ ]:
#@markdown <h3>若启动错误，请运行此单元格安装java1.8<h>
!sudo apt-get remove openjdk*
!sudo apt-get install openjdk-8-jdk
from IPython.display import HTML, clear_output
clear_output()
!java -version

## **其他配置**

In [ ]:
#title **运行此单元格以远程连接colab**
#@markdown <h3>运行此单元格以远程连接colab<h>
import random, string
#password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))
password = 'cruisetian'

#Download ngrok
! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip
#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
! echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc
! echo "export LD_LIBRARY_PATH" >> /root/.bashrc

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')

#Ask token
print("Copy authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
#Print root password
print("Root password: {}".format(password))
#Get public address
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"